In [1]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install nltk


Note: you may need to restart the kernel to use updated packages.


In [5]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from nltk.tokenize import word_tokenize
from collections import Counter
import torch.optim as optim





In [6]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Gauri\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [7]:
import fitz  

doc = fitz.open("text.pdf")
text = ""

for page in doc:
    text += page.get_text()

from nltk.tokenize import word_tokenize

tokens = word_tokenize(text.lower())
tokens = [word for word in tokens if word.isalpha()]

print(tokens[:20])

['thinking', 'through', 'community', 'navigating', 'precarity', 'in', 'the', 'print', 'trade', 'matt', 'ryan', 'of', 'english', 'literature', 'language', 'and', 'linguistics', 'newcastle', 'university', 'newcastle']


In [8]:
print(tokens)

['thinking', 'through', 'community', 'navigating', 'precarity', 'in', 'the', 'print', 'trade', 'matt', 'ryan', 'of', 'english', 'literature', 'language', 'and', 'linguistics', 'newcastle', 'university', 'newcastle', 'uk', 'to', 'abstract', 'in', 'the', 'cutthroat', 'world', 'of', 'the', 'late', 'print', 'trade', 'reputations', 'and', 'livelihoods', 'were', 'precarious', 'shaped', 'by', 'an', 'increasingly', 'restrictive', 'regulatory', 'system', 'the', 'trade', 's', 'most', 'lucrative', 'cial', 'and', 'economic', 'capital', 'was', 'left', 'in', 'the', 'hands', 'of', 'the', 'period', 's', 'most', 'powerful', 'print', 'agents', 'this', 'left', 'those', 'on', 'the', 'outside', 'discontented', 'and', 'the', 'privileged', 'few', 'with', 'a', 'desperate', 'fight', 'to', 'defend', 'their', 'interests', 'consequently', 'whether', 'seeking', 'to', 'gain', 'or', 'maintain', 'capital', 'all', 'those', 'connected', 'to', 'the', 'print', 'trade', 'were', 'involved', 'in', 'the', 'same', 'financial'

In [7]:
vocab = sorted(set(tokens))
word2idx = {w: i + 1 for i, w in enumerate(vocab)}  # Use word2idx consistently
idx2word = {i: w for w, i in word2idx.items()}


In [8]:
print(word2idx)

{'a': 1, 'abc': 2, 'abcs': 3, 'able': 4, 'abolishing': 5, 'about': 6, 'absence': 7, 'abstract': 8, 'absurdity': 9, 'abus': 10, 'abused': 11, 'access': 12, 'accessed': 13, 'according': 14, 'account': 15, 'accumulate': 16, 'accumulating': 17, 'accuracy': 18, 'accused': 19, 'accuses': 20, 'acerbic': 21, 'acquaintance': 22, 'acquire': 23, 'acquiring': 24, 'across': 25, 'acute': 26, 'ad': 27, 'addressed': 28, 'addressing': 29, 'adds': 30, 'admires': 31, 'advance': 32, 'advances': 33, 'advancing': 34, 'afar': 35, 'affaires': 36, 'afford': 37, 'afloat': 38, 'after': 39, 'again': 40, 'against': 41, 'age': 42, 'agent': 43, 'agents': 44, 'agreeing': 45, 'agreements': 46, 'ahead': 47, 'aim': 48, 'aims': 49, 'alan': 50, 'aldersgate': 51, 'alexandra': 52, 'align': 53, 'aligning': 54, 'all': 55, 'alliances': 56, 'allowed': 57, 'allowing': 58, 'allows': 59, 'allure': 60, 'almanacs': 61, 'almost': 62, 'alongside': 63, 'alphabets': 64, 'already': 65, 'also': 66, 'always': 67, 'ambitious': 68, 'amending

In [9]:
print(word2idx.get("researchers", 0))

0


In [10]:
idx2word = {i: w for w, i in word2idx.items()}

In [11]:
vocab_size = len(word2idx) + 1

In [12]:
sequence_length = 3


In [13]:
sequences = []  #inittializing the slicing model process? which is to process sequence data by getting a fixed text or input

for i in range(len(tokens) - sequence_length): #  we substract it here to prevent index out of bound so in the target i + length we are still in loop
    
    seq = tokens[i:i + sequence_length]
    target = tokens[i + sequence_length]
    sequences.append((
        [word2idx.get(word, 0) for word in seq],   # get index or 0 if not found
        word2idx.get(target, 0)                    # same for target
        # word, 0 to find the index of word in the dictionary if it is not found return 0
    ))


    
     
    
    


In [14]:
print(seq)

['library', 'user', 'on']


In [15]:
print(target)

june


In [16]:
print(word2idx[target])

986


In [13]:
# a customized dataset for pytorch



class WordDataset(Dataset):
    def __init__(self, sequences):
     self.sequences = [torch.tensor(x[0]) for x in sequences]
     self.targets = [torch.tensor(x[1]) for x in sequences]

    def __len__(self):
     return len(self.sequences)

    def __getitem__(self, idx):
        return self.sequences[idx], self.targets[idx]
    

    
    

In [18]:
dataset = WordDataset(sequences)
dataLoader = DataLoader(dataset, batch_size=2, shuffle=True)

In [19]:
    print(sequences[:2])

[([1880, 1897, 353], 1200), ([1897, 353, 1200], 1400)]


In [20]:
# defining LSTM MODEL

In [21]:
class NextWordLSTM(nn.Module):
     def __init__(self, vocab_size, embed_dim, hidden_dim):
         #self, input, output, hidden_size
         super().__init__()
         self.embed = nn.Embedding(vocab_size, embed_dim)
         self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
         self.fc = nn.Linear(hidden_dim, vocab_size)

     def forward(self, x):
         x = self.embed(x)
         out, _ = self.lstm(x)         # out: (batch_size, seq_len, hidden_dim)
         out = self.fc(out[:, -1, :])  # (batch_size, hidden_dim) → (batch_size, vocab_size)
         return out

In [22]:
# testing model
# we need to check how wrong it can go calculate loss using criterion
# optimizers lik Adam to make it smarter 
# traininh loop and then core training code


In [ ]:
# HYPERPARAMETER TO DEFINE THE UPS AND DOWNS USED WHEN INITIALIZING THE MODEL
loss_history = []

vocab_size = len(word2idx)+1 
embed_dim = 64
hidden_dim = 128
epochs = 208

# initializing the model
model = NextWordLSTM(vocab_size, embed_dim, hidden_dim)

# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# training loop
for epoch in range(epochs):
    model.train()
    total_loss = 0

    for x_batch, y_batch in dataLoader:
        optimizer.zero_grad()
        outputs = model(x_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}") 

Epoch 1, Loss: 27987.5311
Epoch 2, Loss: 22725.2058
Epoch 3, Loss: 18739.5842
Epoch 4, Loss: 16228.4326
Epoch 5, Loss: 14757.2790
Epoch 6, Loss: 13811.4901
Epoch 7, Loss: 13102.9087
Epoch 8, Loss: 12600.5499
Epoch 9, Loss: 12101.6681
Epoch 10, Loss: 12046.7844
Epoch 11, Loss: 11579.0902
Epoch 12, Loss: 11451.1244
Epoch 13, Loss: 11242.8691
Epoch 14, Loss: 11035.3507
Epoch 15, Loss: 10759.0503
Epoch 16, Loss: 10499.3344
Epoch 17, Loss: 10119.8158
Epoch 18, Loss: 10008.2090
Epoch 19, Loss: 9709.9396
Epoch 20, Loss: 9864.6318
Epoch 21, Loss: 10069.9308
Epoch 22, Loss: 10122.0472
Epoch 23, Loss: 9857.3810
Epoch 24, Loss: 9234.3786
Epoch 25, Loss: 9022.6139
Epoch 26, Loss: 8643.0999
Epoch 27, Loss: 9241.6739
Epoch 28, Loss: 9210.9166
Epoch 29, Loss: 8915.8254
Epoch 30, Loss: 8662.8579
Epoch 31, Loss: 8558.6721
Epoch 32, Loss: 8493.5896


In [ ]:
def predict_next_word(model, seed_text, num_predict=1):
    model.eval()#evaluation mode
    words = word_tokenize(seed_text.lower())
    words = [w for w in words if w.isalpha()] #clead and lowercase

    for _ in range(num_predict):
        x = [word2idx.get(w, 0) for w in words[-sequence_length:]]  # use last N words
        x = torch.tensor(x).unsqueeze(0)  # batch shape: (1, sequence_length)
        
        output = model(x)
        predicted_idx = output.argmax(dim=1).item()
        predicted_word = idx2word.get(predicted_idx, "<unk>")

        words.append(predicted_word)

    return " ".join(words)

In [ ]:
print(predict_next_word(model, "applications to", num_predict=2))

In [ ]:
# ADD A  NOVEL TO IMPROVE DATA ACCURACY AND UPLOAD ON GITHUB

In [ ]:
# RNN MODEL CLASS 
class NextWordRNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embed_dim) 
        self.rnn = nn.RNN(hidden_dim, embed_dim, batch_first = True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

   def __init__(self, x):
       x = self.embed(x)            # (batch, seq_len, embed_dim)
       out, _ = self.rnn(x)         # out: (batch, seq_len, hidden_dim)
       out = self.fc(out[:, -1, :]) # only last timestep's output
       return out
        

In [ ]:
# CNN MODEL
#   super(SimpleCNN, self).__init__()
    #     self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
    #     self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
    #     self.pool = nn.MaxPool2d(2, 2)
    #     self.fc1 = nn.Linear(32 * 8 * 8, 128)
    #     self.fc2 = nn.Linear(128, 10)  # CIFAR-10 has 10 classes

    # def forward(self, x):
    #     x = self.pool(F.relu(self.conv1(x)))  # (B, 16, 16, 16)
    #     x = self.pool(F.relu(self.conv2(x)))  # (B, 32, 8, 8)
    #     x = x.view(x.size(0), -1)             # flatten
    #     x = F.relu(self.fc1(x))
    #     x = self.fc2(x)
    #     return x